In [100]:
import pandas as pd
import numpy as np
import datetime as datetime

In [101]:
policy_df = pd.read_csv('./data/processed/policy_clean.csv',usecols = ['policy_id','policy_start_date','policy_end_date','vehicle_age','region','channel','sum_insured','premium','exposure'])
claims_df= pd.read_csv('./data/processed/claims_clean.csv',usecols = ['policy_id','claim_id','accident_date','claim_amount','development_year'])
payment_df = pd.read_csv('./data/processed/payment_clean.csv')


In [102]:
policy_df.dtypes


policy_id              int64
policy_start_date     object
policy_end_date       object
vehicle_age            int64
region                object
channel               object
sum_insured            int64
premium              float64
exposure             float64
dtype: object

In [103]:
claims_df.dtypes

policy_id           float64
claim_id            float64
accident_date        object
development_year      int64
claim_amount        float64
dtype: object

In [104]:
# Convert date columns to datetime format
policy_df['policy_start_date'] = pd.to_datetime(policy_df['policy_start_date'], errors='coerce')
policy_df['policy_end_date'] = pd.to_datetime(policy_df['policy_end_date'],  errors='coerce')
claims_df['accident_date'] = pd.to_datetime(claims_df['accident_date'], errors='coerce')

In [105]:
# merge policy And claims data on policy number
merged_df = pd.merge(policy_df,claims_df, on='policy_id', how='left')
merged_df.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure', 'claim_id',
       'accident_date', 'development_year', 'claim_amount'],
      dtype='object')

In [106]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498778 entries, 0 to 498777
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   policy_id          498778 non-null  int64         
 1   policy_start_date  498778 non-null  datetime64[ns]
 2   policy_end_date    498778 non-null  datetime64[ns]
 3   vehicle_age        498778 non-null  int64         
 4   region             498778 non-null  object        
 5   channel            498778 non-null  object        
 6   sum_insured        498778 non-null  int64         
 7   premium            497277 non-null  float64       
 8   exposure           498778 non-null  float64       
 9   claim_id           44289 non-null   float64       
 10  accident_date      44289 non-null   datetime64[ns]
 11  development_year   44289 non-null   float64       
 12  claim_amount       44289 non-null   float64       
dtypes: datetime64[ns](3), float64(5), int64(3), 

In [107]:
#valid claims within policy period and invalid claims outside policy period
valid_claims = merged_df[
    (merged_df['accident_date'] >= merged_df['policy_start_date']) &
    (merged_df['accident_date'] <= merged_df['policy_end_date'])]

invalid_claims = merged_df[
    (merged_df['accident_date'] < merged_df['policy_start_date']) |
    (merged_df['accident_date'] > merged_df['policy_end_date'])]


In [108]:
# Link valid claims with exposure table
linked_valid_claims=valid_claims.copy()
linked_valid_claims

,policy_id,policy_start_date,policy_end_date,vehicle_age,region,channel,sum_insured,premium,exposure,claim_id,accident_date,development_year,claim_amount
23,24,2024-06-26,2025-06-13,9,South,Direct,579900,10275.0,0.966461,1.0,2025-06-06,0.0,521910.0
31,32,2019-08-04,2020-08-19,2,Central,Online,2658400,41022.0,1.000000,2.0,2019-10-24,1.0,92636.0
45,46,2024-10-17,2025-10-24,10,East,Agent,836800,18007.0,1.000000,3.0,2024-11-12,1.0,28492.0
54,55,2021-06-26,2022-06-01,5,West,Online,270800,4314.0,0.933607,4.0,2022-05-28,3.0,17377.0
55,55,2021-06-26,2022-06-01,5,West,Online,270800,4314.0,0.933607,5.0,2021-09-14,0.0,13725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498710,499932,2018-01-31,2019-01-22,15,East,Online,478400,10580.0,0.977413,44582.0,2018-04-02,0.0,229361.0
498733,499955,2021-09-24,2022-08-15,15,Unknown,Broker,2867000,64604.0,0.892539,44584.0,2021-12-05,2.0,249877.0
498766,499989,2021-09-18,2022-10-08,0,North,Online,492400,7927.0,1.000000,44585.0,2022-04-07,1.0,4051.0
498773,499996,2018-02-20,2019-02-03,4,East,Direct,854300,13490.0,0.955510,44586.0,2018-03-22,1.0,183395.0


In [109]:
# Calculate claim frequency
claim_counts = linked_valid_claims.groupby('policy_id').size().reset_index(name='claim_count')
frequency_df = pd.merge(linked_valid_claims, claim_counts, on='policy_id', how='left')

In [110]:
claim_counts

,policy_id,claim_count
0,24,1
1,32,1
2,46,1
3,55,2
4,68,1
...,...,...
41905,499932,2
41906,499955,1
41907,499989,1
41908,499996,1


In [111]:
frequency_df

,policy_id,policy_start_date,policy_end_date,vehicle_age,region,channel,sum_insured,premium,exposure,claim_id,accident_date,development_year,claim_amount,claim_count
0,24,2024-06-26,2025-06-13,9,South,Direct,579900,10275.0,0.966461,1.0,2025-06-06,0.0,521910.0,1
1,32,2019-08-04,2020-08-19,2,Central,Online,2658400,41022.0,1.000000,2.0,2019-10-24,1.0,92636.0,1
2,46,2024-10-17,2025-10-24,10,East,Agent,836800,18007.0,1.000000,3.0,2024-11-12,1.0,28492.0,1
3,55,2021-06-26,2022-06-01,5,West,Online,270800,4314.0,0.933607,4.0,2022-05-28,3.0,17377.0,2
4,55,2021-06-26,2022-06-01,5,West,Online,270800,4314.0,0.933607,5.0,2021-09-14,0.0,13725.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44062,499932,2018-01-31,2019-01-22,15,East,Online,478400,10580.0,0.977413,44582.0,2018-04-02,0.0,229361.0,2
44063,499955,2021-09-24,2022-08-15,15,Unknown,Broker,2867000,64604.0,0.892539,44584.0,2021-12-05,2.0,249877.0,1
44064,499989,2021-09-18,2022-10-08,0,North,Online,492400,7927.0,1.000000,44585.0,2022-04-07,1.0,4051.0,1
44065,499996,2018-02-20,2019-02-03,4,East,Direct,854300,13490.0,0.955510,44586.0,2018-03-22,1.0,183395.0,1


In [112]:
frequency_df['policy_year'] = frequency_df['policy_start_date'].dt.year
frequency_df.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure', 'claim_id',
       'accident_date', 'development_year', 'claim_amount', 'claim_count',
       'policy_year'],
      dtype='object')

In [113]:
frequency_dfs = frequency_df[['policy_id','claim_id' ,'exposure', 'claim_count', 'vehicle_age', 'region', 'channel', 'sum_insured','policy_year']].copy()

In [121]:
#frequency data and severity data export for GLM Inputs
frequency_dfs.to_csv('./data/processed/frequency_data.csv', index=False)
severity_df = linked_valid_claims[['policy_id','claim_id','accident_date', 'claim_amount', 'vehicle_age', 'region', 'channel', 'sum_insured']]
severity_df.to_csv('./data/processed/severity_data.csv', index=False)

In [115]:
#reserving triangle
rt_df = pd.merge(frequency_df,payment_df[['claim_id','paid_amount','case_reserve']], on='claim_id', how='left')
rt_df.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure', 'claim_id',
       'accident_date', 'development_year', 'claim_amount', 'claim_count',
       'policy_year', 'paid_amount', 'case_reserve'],
      dtype='object')

In [116]:
rt_df['accident_year']= rt_df['accident_date'].dt.year
rt_df.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure', 'claim_id',
       'accident_date', 'development_year', 'claim_amount', 'claim_count',
       'policy_year', 'paid_amount', 'case_reserve', 'accident_year'],
      dtype='object')

In [117]:
reserving_df = rt_df[['accident_year','development_year','paid_amount','case_reserve']]


In [120]:
reserving_df.to_csv('./data/processed/reserving_triangle_data.csv', index=False )